## Text Feature Extractor
this notebook generates for each text some additional/handcrafted features characterizing its named entity composition and their relative relative position (entity distance):distance is the number of words separating 2 entities (it can be negative)
* number of sentences
* number of words
* distinct count of drug name entities
* distinct count of active ingredient entities
* count of question marks (typically to identify specifically multi-intent label)
* individual count of interrogative pronoun entities (one column per pronoun: quand, qui, quoi, ou, comment, pourquoi, combien, quel(s|le,..)
* distinct count of time entities (eg: jours, après midi, soir, année, 12h, mardi, samedi, temps....)
* distinct count of quantity entities (eg: 5mg, 10ml, ...)
* count of association entities (eg: et, avec, ou, ...)
* distance between interrogative pronoun and drug name entities
* distance between active ingredient and drug name entities
* distance between quantity and drug name entities
* distance between time and drug name entities
* distance between question marks and drug name entities

these extracted features are stored into this [file](../../data/staging_data/text_extracted_features.csv) with ID column to join it with the former train dataset if required.

In [1]:
import pandas as pd
import numpy as np

XTrain = pd.read_csv('../../data/staging_data/mispelling_fixed_clean_input_train.csv', sep=',')

In [2]:
from collections import Counter
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def words(text):
    return re.findall(r'\w+', text.lower())

# value domain based named entities
drugNames = set(words(open('../../data/staging_data/drug_names.txt').read()))
ingredientNames = set(words(open('../../data/staging_data/ingredients.txt').read()))

# reg expression based named entities
questionMarkRegEx = r"[\?]+"
sentenceSeparatorRegEx = r"[\?]+|!|:|;|[\.]+"
#quiRegEx = r"[\s\?\.\!\:\;]?[Q|q]ui\s" qui term is ambiguous
combienRegEx = r"[\s\?\.\!\:\;]?[C|c]ombien\s"
pourquoiRegEx = r"[\s\?\.\!\:\;]?[P|p]ourquoi\s"
quandRegEx = r"[\s\?\.\!\:\;]?[Q|q]uand\s"
quoiRegEx = r"[\s\?\.\!\:\;]?[Q|q]uoi\s|[\s\?\.\!\:\;]?[Q|q]uel[l|s|le|les]?\s"
commentRegEx = r"[\s\?\.\!\:\;]?[C|c]omment\s"


# gold standard
timeRegEx = r"((D|d)emain|(H|h)ier|(M|m)atin|(M|m)idi|(S|s)oir|mois|jours|semaine(s|S)|heure(s|S))( )*( |,|\.)"
quantityRegEx = r"[0-9]+\s?[m|k]?g|\s[m|k]?g[^a-zA-Zéàèî]|[0-9]+\s?[m|c]?l|\s[m|c]?l[^a-zA-Zéàèî]"

def getWordCount(text):
    words = word_tokenize(text, language='french')
    return len(words)

def getEntityCount(text, entityValueSet, lowerCase):
    ''' return distinct count of entity occurrences found from the text'''
    words = word_tokenize(text, language='french')
    if lowerCase:
        words = map(lambda x: x.lower(), words)
    return len(list(set(words) & entityValueSet))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jacques\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
XTrain['drugCount'] = XTrain['question'].map(lambda text: getEntityCount(text, drugNames, True))
XTrain['ingredientCount'] = XTrain['question'].map(lambda text: getEntityCount(text, ingredientNames, True))

In [4]:
XTrain['timeCount'] = XTrain['question'].map(lambda text : len(re.findall(timeRegEx, text)))
XTrain['quantitiesCount'] = XTrain['question'].map(lambda text : len(re.findall(quantityRegEx, text)))
XTrain['questionMarkCount'] = XTrain['question'].map(lambda text : len(re.findall(questionMarkRegEx, text)))
XTrain['sentenceCount'] = XTrain['question'].map(lambda text : 1 + len(re.findall(sentenceSeparatorRegEx, text)))
XTrain['wordCount'] = XTrain['question'].map(lambda text : getWordCount(text))

In [5]:
XTrain['combienCount'] = XTrain['question'].map(lambda text : len(re.findall(combienRegEx, text)))
XTrain['pourquoiCount'] = XTrain['question'].map(lambda text : len(re.findall(pourquoiRegEx, text)))
XTrain['quandCount'] = XTrain['question'].map(lambda text : len(re.findall(quandRegEx, text)))
XTrain['quoiCount'] = XTrain['question'].map(lambda text : len(re.findall(quoiRegEx, text)))
XTrain['commentCount'] = XTrain['question'].map(lambda text : len(re.findall(commentRegEx, text)))


In [6]:
pd.set_option("display.max_colwidth",20000)
XTrain.head(2)

,ID,question,drugCount,ingredientCount,timeCount,quantitiesCount,questionMarkCount,sentenceCount,wordCount,combienCount,pourquoiCount,quandCount,quoiCount,commentCount
0,0,"bonjour, trompé forum question alors repose ici. pris première fois hier paroxétine matin catastrophe. picotement dasn tous corps annonciateur sueur froide très très massive vomissement. deux crises depuis heure mat. cela semble passer mains reste moites chaude estce normal première fois merci tous",0,1,2,0,0,5,48,0,0,0,0,0
1,1,motilium soulagera contre nausées,1,0,0,0,0,1,4,0,0,0,0,0


In [7]:
XTrain.columns

Index(['ID', 'question', 'drugCount', 'ingredientCount', 'timeCount',
       'quantitiesCount', 'questionMarkCount', 'sentenceCount', 'wordCount',
       'combienCount', 'pourquoiCount', 'quandCount', 'quoiCount',
       'commentCount'],
      dtype='object')

In [8]:
# save the extraction result
XTrain.drop(["question"], inplace=True, axis=1)
XTrain.to_csv("../../data/staging_data/text_extracted_features.csv", index=None)